In [1]:
import pandas as pd
import numpy as np
import os
import glob
import random

import warnings
warnings.filterwarnings("ignore")

In [2]:
def preprocess_data(data, is_train=True):
    
    temp = data.copy()
    temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]

    if is_train==True:          
    
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill')
        temp = temp.dropna()
        
        return temp.iloc[:-96]

    elif is_train==False:
        
        temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]
                              
        return temp.iloc[-48:, :]

In [4]:
train = pd.read_csv('./data/train/train.csv')

test = []

for i in range(81):
    file_path = './data/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    test.append(temp)

df_test = pd.concat(test)

submission = pd.read_csv('./data/sample_submission.csv')

In [5]:
df_train = preprocess_data(train)

df_train.shape, df_test.shape

((52464, 9), (3888, 7))

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_train[['DHI','DNI','WS','RH','T']] = scaler.fit_transform(df_train[['DHI','DNI','WS','RH','T']])
df_test[['DHI','DNI','WS','RH','T']] = scaler.fit_transform(df_test[['DHI','DNI','WS','RH','T']])

In [7]:
Day  = df_train.iloc[:, :-2]
Day7 = df_train.iloc[:, -2]
Day8 = df_train.iloc[:, -1]

In [8]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras import layers
from tensorflow.keras.backend import mean, maximum

In [9]:
def quantile_loss(q, y, pred):
    err = (y-pred)
    return mean(maximum(q*err, (q-1)*err), axis=-1)

In [10]:
q_lst = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [22]:
model7 = tf.keras.Sequential([
    layers.Dense(52, activation='relu', input_shape=(Day.shape)),
    layers.Dense(39, activation='relu'),
    layers.Dense(26, activation='relu'),
    layers.Dense(1)
])

In [24]:
results7 = pd.DataFrame()

for q in q_lst:
    model7.compile(loss=lambda y,pred: quantile_loss(q,y,pred), optimizer='adam')
    model7.fit(Day, Day7, epochs=25, batch_size=48, validation_split=0.25)
    pred = pd.DataFrame(model7.predict(df_test))
    results7 = pd.concat([results7, pred], axis=1)

results7.shape

Epoch 1/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3339 - val_loss: 1.5714
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3348 - val_loss: 1.5647
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3244 - val_loss: 1.6241
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3325 - val_loss: 1.5917
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3308 - val_loss: 1.5631
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3273 - val_loss: 1.5906
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3359 - val_loss: 1.5658
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3161 - val_loss: 1.5744
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3297 - val_loss: 1.5826
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3239 - val_loss: 1.5700

820/820 [==============================] - 1s 1ms/step - loss: 2.4893 - val_loss: 3.1524
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4956 - val_loss: 3.0434
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4681 - val_loss: 3.0718
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4457 - val_loss: 3.0369
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4423 - val_loss: 3.0970
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4504 - val_loss: 3.0372
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4652 - val_loss: 3.0323
Epoch 8/25
820/820 [==============================] - 1s 2ms/step - loss: 2.4633 - val_loss: 2.9948
Epoch 9/25
820/820 [==============================] - 1s 2ms/step - loss: 2.4658 - val_loss: 3.0303
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4559 - val_loss: 3.0257
Epoch 11/2

820/820 [==============================] - 1s 1ms/step - loss: 2.4555 - val_loss: 3.0513
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4606 - val_loss: 3.1058
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4037 - val_loss: 3.0326
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3757 - val_loss: 3.0479
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4495 - val_loss: 3.0341
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4109 - val_loss: 3.0539
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4451 - val_loss: 3.0661
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3799 - val_loss: 3.0341
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4124 - val_loss: 3.0618
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4251 - val_loss: 3.0179
Epoch 12/

820/820 [==============================] - 1s 1ms/step - loss: 1.7643 - val_loss: 2.2356
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7285 - val_loss: 2.2313
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7481 - val_loss: 2.2952
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7528 - val_loss: 2.2642
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7380 - val_loss: 2.2693
Epoch 12/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7457 - val_loss: 2.2738
Epoch 13/25
820/820 [==============================] - 1s 2ms/step - loss: 1.7331 - val_loss: 2.2680
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7242 - val_loss: 2.2304
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7296 - val_loss: 2.2166
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7428 - val_loss: 2.2790
Epoc

820/820 [==============================] - 1s 1ms/step - loss: 0.7221 - val_loss: 0.9340
Epoch 13/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7286 - val_loss: 0.9473
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7216 - val_loss: 0.9560
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7094 - val_loss: 0.9301
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7108 - val_loss: 0.9781
Epoch 17/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7166 - val_loss: 0.9670
Epoch 18/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7108 - val_loss: 0.9204
Epoch 19/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7060 - val_loss: 0.9376
Epoch 20/25
820/820 [==============================] - 1s 2ms/step - loss: 0.7280 - val_loss: 0.9439
Epoch 21/25
820/820 [==============================] - 1s 1ms/step - loss: 0.6928 - val_loss: 0.9704
Ep

(3888, 9)

In [25]:
model8 = tf.keras.Sequential([
    layers.Dense(52, activation='relu', input_shape=(Day.shape)),
    layers.Dense(39, activation='relu'),
    layers.Dense(26, activation='relu'),
    layers.Dense(1)
])

In [26]:
results8 = pd.DataFrame()

for q in q_lst:
    model8.compile(loss=lambda y,pred: quantile_loss(q,y,pred), optimizer='adam')
    model8.fit(Day, Day8, epochs=25, batch_size=48, validation_split=0.25)
    pred = pd.DataFrame(model8.predict(df_test))
    results8 = pd.concat([results8, pred], axis=1)

results8.shape

Epoch 1/25
820/820 [==============================] - 2s 2ms/step - loss: 1.4617 - val_loss: 1.6463
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4620 - val_loss: 1.6404
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4301 - val_loss: 1.6451
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4433 - val_loss: 1.6452
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4257 - val_loss: 1.6332
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4186 - val_loss: 1.6272
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4212 - val_loss: 1.6270
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4050 - val_loss: 1.6138
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4214 - val_loss: 1.6119
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4189 - val_loss: 1.6016

820/820 [==============================] - 1s 1ms/step - loss: 2.6488 - val_loss: 3.1061
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6488 - val_loss: 3.1129
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5914 - val_loss: 3.1114
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6525 - val_loss: 3.1583
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6329 - val_loss: 3.0768
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6353 - val_loss: 3.1324
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6460 - val_loss: 3.1336
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6566 - val_loss: 3.3266
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6369 - val_loss: 3.1441
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6391 - val_loss: 3.1102
Epoch 11/2

820/820 [==============================] - 1s 1ms/step - loss: 2.5585 - val_loss: 3.2017
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5894 - val_loss: 3.1845
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5302 - val_loss: 3.1556
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6138 - val_loss: 3.1509
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5777 - val_loss: 3.1098
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6202 - val_loss: 3.1369
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5639 - val_loss: 3.1438
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5591 - val_loss: 3.1522
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5963 - val_loss: 3.1307
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5651 - val_loss: 3.2028
Epoch 12/

820/820 [==============================] - 1s 1ms/step - loss: 1.8681 - val_loss: 2.3063
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8347 - val_loss: 2.3504
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8253 - val_loss: 2.3203
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8418 - val_loss: 2.3088
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8790 - val_loss: 2.2941
Epoch 12/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8419 - val_loss: 2.3409
Epoch 13/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8854 - val_loss: 2.3399
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8366 - val_loss: 2.3002
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8077 - val_loss: 2.3209
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8638 - val_loss: 2.3154
Epoc

820/820 [==============================] - 1s 1ms/step - loss: 0.7425 - val_loss: 0.9301
Epoch 13/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7337 - val_loss: 0.9441
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7361 - val_loss: 0.9473
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7449 - val_loss: 0.9270
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7349 - val_loss: 0.9468
Epoch 17/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7350 - val_loss: 0.9335
Epoch 18/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7369 - val_loss: 0.9393
Epoch 19/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7229 - val_loss: 0.9192
Epoch 20/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7298 - val_loss: 0.9509
Epoch 21/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7315 - val_loss: 0.9241
Ep

(3888, 9)

In [27]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results7.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results8.sort_index().values
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,-0.008738,-0.004625,0.004559,-0.002137,0.001333,0.004818,0.003396,0.008665,0.004672
1,0.csv_Day7_0h30m,-0.008892,-0.004596,0.004669,-0.002201,0.001351,0.005070,0.003445,0.008758,0.004697
2,0.csv_Day7_1h00m,-0.009784,-0.004373,0.004370,-0.002104,0.001294,0.005312,0.003260,0.008324,0.004567
3,0.csv_Day7_1h30m,-0.009643,-0.004409,0.004324,-0.002051,0.001281,0.004983,0.003280,0.008274,0.004552
4,0.csv_Day7_2h00m,-0.010182,-0.005545,0.003899,-0.001914,0.001204,0.005322,0.003062,0.007712,0.004406
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,-0.007796,0.002805,-0.009455,-0.000738,0.004431,0.001312,0.006316,-0.003241,0.005092
7772,80.csv_Day8_22h00m,-0.008310,0.003021,-0.010354,-0.001192,0.004645,0.001454,0.007140,-0.003547,0.005847
7773,80.csv_Day8_22h30m,-0.008386,0.003025,-0.010444,-0.001282,0.004647,0.001467,0.007208,-0.003581,0.005888
7774,80.csv_Day8_23h00m,-0.008866,0.003252,-0.011321,-0.001689,0.004882,0.001596,0.008027,-0.003897,0.006652


In [28]:
submission.to_csv('./submission/submission_20210113-1.csv', index=False)